In [35]:
import numpy as np
import pandas as pd
import emoji

In [152]:
X = pd.read_csv("../Datasets/emojified_text/emojify_train_x.csv", header=None, sep="' '")
Y = pd.read_csv("../Datasets/emojified_text/Emojify_Y_train.csv", header=None, sep=' ')

Xt = pd.read_csv("../Datasets/emojified_text/emojiy_test_x.csv", header=None, sep="' '")
Yt = pd.read_csv("../Datasets/emojified_text/emojiy_y_test.csv", header=None, sep=' ')

<ipython-input-152-e34972428f84>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  X = pd.read_csv("../Datasets/emojified_text/emojify_train_x.csv", header=None, sep="' '")
<ipython-input-152-e34972428f84>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  Xt = pd.read_csv("../Datasets/emojified_text/emojiy_test_x.csv", header=None, sep="' '")


In [248]:
Xtrain = np.asarray(X, dtype=str).reshape((-1,1))
Ytrain = np.asarray(Y, dtype=int).reshape((-1,1))

Xtest = np.asarray(Xt, dtype=str).reshape((-1,1))
Ytest = Yt.values

In [98]:
embedding_idx = {}

with open("../Datasets/glove.6B.50d.txt/glove.6B.50d.txt", encoding='utf-8') as f:
    for line in f:
        word_values = line.split()
        word = word_values[0]
        values = np.asarray(word_values[1:], dtype='float')
        embedding_idx[word] = values

In [83]:
emoji_dict = {
    "0": ":red_heart:",
    "1": ":baseball:",
    "2": ":grinning_face_with_big_eyes:",
    "3": ":disappointed_face:",
    "4": ":fork_and_knife:"
}

In [85]:
emoji.emojize(emoji_dict["0"])

'❤️'

In [218]:
# data/sentences in embedding format
def data_embedding(X):
    datasize = X.shape[0]
    maxlen = 10 # sentences will be clipped after 10 words
    
    embed_data = list()
    for sent in X:
        sent = sent.split()
        embed_sent = list()
        for w_idx in range(maxlen):
            try:
                word = sent[w_idx].lower()
                embed_sent.append(embedding_idx[word])
            except:
                embed_sent.append(np.zeros((50,)))
                
        embed_data.append(np.asarray(embed_sent))
    
    return np.asarray(embed_data)

In [235]:
Xtrain_emb = data_embedding(Xtrain[:,0])
Xtest_emb = data_embedding(Xtest[:,0])

In [234]:
Xtrain[0,0]

"'never talk to me again"

Define RNN/LSTM model

In [265]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

In [275]:
model = Sequential()
model.add(LSTM(64, input_shape=(10, 50)))
model.add(Dropout(0.2))
model.add(Dense(28,  activation='relu'))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 64)                29440     
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 28)                1820      
                                                                 
 dense_10 (Dense)            (None, 5)                 145       
                                                                 
 activation_2 (Activation)   (None, 5)                 0         
                                                                 
Total params: 31,405
Trainable params: 31,405
Non-trainable params: 0
_________________________________________________________________


In [267]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

earlystop = EarlyStopping(monitor='val_accuracy', mode='max', patience=2)
checkpoint = ModelCheckpoint("sentiment-analysis-model.h5", verbose=1, monitor='val_accuracy', mode='max', save_weights_only=True, )

hist = model.fit(Xtrain_emb, Ytrain, batch_size=10, epochs=20, callbacks=[earlystop, checkpoint], use_multiprocessing=True)

Epoch 1/20


ValueError: in user code:

    File "Z:\Google\Anaconda\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "Z:\Google\Anaconda\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "Z:\Google\Anaconda\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "Z:\Google\Anaconda\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "Z:\Google\Anaconda\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "Z:\Google\Anaconda\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "Z:\Google\Anaconda\lib\site-packages\

In [251]:
Ytrain.shape

(132, 1)

In [252]:
Xtrain_emb.shape

(132, 10, 50)

In [253]:
# we need to convert Ytrain into one-hot format
from keras.utils import to_categorical

Ytrain_oht = to_categorical(Ytrain)
Ytest_oht = to_categorical(Ytest)

In [276]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

earlystop = EarlyStopping(monitor='val_accuracy', mode='max', patience=20)
checkpoint = ModelCheckpoint("sentiment-analysis-model.h5", verbose=1, monitor='val_accuracy', mode='max', save_weights_only=True, save_best_only=True)

hist = model.fit(Xtrain_emb, Ytrain_oht, batch_size=24, epochs=100, validation_split=0.2, callbacks=[earlystop, checkpoint], use_multiprocessing=True)

Epoch 1/100
5/5 [==============================] - ETA: 0s - loss: 1.5547 - accuracy: 0.3333
Epoch 1: val_accuracy improved from -inf to 0.25926, saving model to sentiment-analysis-model.h5
5/5 [==============================] - 34s 749ms/step - loss: 1.5547 - accuracy: 0.3333 - val_loss: 1.6162 - val_accuracy: 0.2593
Epoch 2/100
5/5 [==============================] - ETA: 0s - loss: 1.4831 - accuracy: 0.3714
Epoch 2: val_accuracy did not improve from 0.25926
5/5 [==============================] - 1s 111ms/step - loss: 1.4831 - accuracy: 0.3714 - val_loss: 1.6211 - val_accuracy: 0.2593
Epoch 3/100
5/5 [==============================] - ETA: 0s - loss: 1.4456 - accuracy: 0.4000
Epoch 3: val_accuracy improved from 0.25926 to 0.29630, saving model to sentiment-analysis-model.h5
5/5 [==============================] - 1s 145ms/step - loss: 1.4456 - accuracy: 0.4000 - val_loss: 1.5646 - val_accuracy: 0.2963
Epoch 4/100
5/5 [==============================] - ETA: 0s - loss: 1.3673 - accuracy:

Epoch 30/100
5/5 [==============================] - ETA: 0s - loss: 0.2267 - accuracy: 0.9143
Epoch 30: val_accuracy improved from 0.70370 to 0.74074, saving model to sentiment-analysis-model.h5
5/5 [==============================] - 1s 124ms/step - loss: 0.2267 - accuracy: 0.9143 - val_loss: 1.0584 - val_accuracy: 0.7407
Epoch 31/100
4/5 [=======================>......] - ETA: 0s - loss: 0.2144 - accuracy: 0.9271
Epoch 31: val_accuracy did not improve from 0.74074
5/5 [==============================] - 0s 98ms/step - loss: 0.2144 - accuracy: 0.9333 - val_loss: 1.0470 - val_accuracy: 0.7037
Epoch 32/100
5/5 [==============================] - ETA: 0s - loss: 0.1390 - accuracy: 0.9619
Epoch 32: val_accuracy did not improve from 0.74074
5/5 [==============================] - 0s 99ms/step - loss: 0.1390 - accuracy: 0.9619 - val_loss: 1.2935 - val_accuracy: 0.5926
Epoch 33/100
5/5 [==============================] - ETA: 0s - loss: 0.2309 - accuracy: 0.9238
Epoch 33: val_accuracy did not imp

In [259]:
Ytrain_oht.shape

(132, 5)

In [264]:
Xtrain_emb.shape

(132, 10, 50)

In [279]:
stack_model = Sequential()
stack_model.add(LSTM(64, input_shape=(10, 50), return_sequences=True))
stack_model.add(LSTM(64, return_sequences=False))
stack_model.add(Dropout(0.2))
stack_model.add(Dense(28,  activation='relu'))
stack_model.add(Dense(5))
stack_model.add(Activation('softmax'))
stack_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
stack_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 10, 64)            29440     
                                                                 
 lstm_12 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 28)                1820      
                                                                 
 dense_14 (Dense)            (None, 5)                 145       
                                                                 
 activation_4 (Activation)   (None, 5)                 0         
                                                                 
Total params: 64,429
Trainable params: 64,429
Non-tra

In [280]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

earlystop = EarlyStopping(monitor='val_accuracy', mode='max', patience=20)
checkpoint = ModelCheckpoint("sentiment-analysis-model.h5", verbose=0, monitor='val_accuracy', mode='max', save_weights_only=True, save_best_only=True)

hist = stack_model.fit(Xtrain_emb, Ytrain_oht, batch_size=24, epochs=100, validation_split=0.5, callbacks=[earlystop, checkpoint], use_multiprocessing=True)

Epoch 1/100
3/3 [==============================] - 22s 2s/step - loss: 1.6052 - accuracy: 0.2273 - val_loss: 1.5975 - val_accuracy: 0.1818
Epoch 2/100
3/3 [==============================] - 1s 289ms/step - loss: 1.5477 - accuracy: 0.3636 - val_loss: 1.5710 - val_accuracy: 0.2576
Epoch 3/100
3/3 [==============================] - 1s 279ms/step - loss: 1.5255 - accuracy: 0.3182 - val_loss: 1.5617 - val_accuracy: 0.2879
Epoch 4/100
3/3 [==============================] - 1s 256ms/step - loss: 1.4969 - accuracy: 0.3485 - val_loss: 1.5386 - val_accuracy: 0.3636
Epoch 5/100
3/3 [==============================] - 0s 192ms/step - loss: 1.4739 - accuracy: 0.3788 - val_loss: 1.5041 - val_accuracy: 0.3485
Epoch 6/100
3/3 [==============================] - 1s 209ms/step - loss: 1.4089 - accuracy: 0.3636 - val_loss: 1.4826 - val_accuracy: 0.3333
Epoch 7/100
3/3 [==============================] - 1s 263ms/step - loss: 1.3217 - accuracy: 0.5000 - val_loss: 1.4175 - val_accuracy: 0.4091
Epoch 8/100
3/3